In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from bracketology_rl import Bracket
from bracketgym1game import TournamentEnv
import warnings
import gym
import numpy as np
from tourney_tools import * 
warnings.filterwarnings('ignore')
import multiprocessing as mp
from treelib import Node,Tree
import igraph as ig
import graphviz

/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_67761/2171227993.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [20]:
years = list(range(2003,2019))
year=years[-1]

bracket = Bracket(year)

team_data=pd.read_csv('Process_data/pm_w_names.csv')
year

2018

In [21]:
env=TournamentEnv(team_data=team_data,discrete=True,loading_bar=True,bust_stop=False,year=year)


TypeError: __init__() got an unexpected keyword argument 'year'

In [30]:
obs = env.reset(season=year,weight_scores=False,track_wins=True)
done = False
while not done:
    actions = np.random.randn(2)

    action=np.argmax(actions)
    data=env.current_data.loc[env.current_step]
    top_team=data.top_team
    bottom_team=data.bottom_team
    res_list=env.result_dict[env.current_step]
    teams=[top_team,bottom_team]
    if top_team in res_list:
        action=0
    else:
        action=1
    win_team=teams[action]
    print(win_team)

    obs, reward, done, info = env.step(round(action))

Villanova
Virginia Tech
West Virginia
Wichita St
Florida
Texas Tech
Arkansas
Purdue
Kansas
Seton Hall
Clemson
Auburn
TCU
Michigan St
Rhode Island
Duke
Virginia
Creighton
Kentucky
Arizona
Miami FL
Tennessee
Nevada
Cincinnati
Xavier
Missouri
Ohio St
Gonzaga
Houston
Michigan
Texas A&M
North Carolina
Villanova
West Virginia
Florida
Purdue
Kansas
Clemson
Michigan St
Rhode Island
Creighton
Kentucky
Tennessee
Nevada
Xavier
Ohio St
Houston
Texas A&M
Villanova
Purdue
Kansas
Rhode Island
Kentucky
Nevada
Ohio St
Texas A&M
Villanova
Kansas
Nevada
Texas A&M
Texas A&M
Villanova
Villanova


In [31]:
tournament=env.action_data
tournament=tournament.sort_values('round',ascending=False).reset_index(drop=True)
tournament


,winning_team,losing_team,round,top_team_proba,bottom_team_proba,top_team,top_seed,bottom_team,bottom_seed,region,winning_seed,losing_seed,upset
0,Villanova,Texas A&M,6,0.0,1.0,Texas A&M,7.0,Villanova,1.0,round6,1.0,7.0,0.0
1,Villanova,Kansas,5,0.0,1.0,Kansas,1.0,Villanova,1.0,round5,1.0,1.0,0.0
2,Texas A&M,Nevada,5,0.0,1.0,Nevada,7.0,Texas A&M,7.0,round5,7.0,7.0,0.0
3,Texas A&M,Ohio St,4,0.0,1.0,Ohio St,5.0,Texas A&M,7.0,West,7.0,5.0,1.0
4,Nevada,Kentucky,4,0.0,1.0,Kentucky,5.0,Nevada,7.0,South,7.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Ohio St,S Dakota St,1,1.0,0.0,Ohio St,5.0,S Dakota St,12.0,West,5.0,12.0,0.0
59,Gonzaga,UNC Greensboro,1,1.0,0.0,Gonzaga,4.0,UNC Greensboro,13.0,West,4.0,13.0,0.0
60,Houston,San Diego St,1,1.0,0.0,Houston,6.0,San Diego St,11.0,West,6.0,11.0,0.0
61,Michigan,Montana,1,1.0,0.0,Michigan,3.0,Montana,14.0,West,3.0,14.0,0.0


In [32]:
g = ig.Graph.Tree(len(tournament), 2)
def index_verts(graph,data):
    g=graph
    for i, v in enumerate(g.vs):
        g.vs[i]['index']=i
        g.vs[i]['name']=data.loc[i,'winning_team']
        g.vs[i]['losing_team']=data.loc[i,'losing_team']
        g.vs[i]['seed']=data.loc[i,'winning_seed']
        g.vs[i]['round']=data.loc[i,'round']
        g.vs[i]['region']=data.loc[i,'region']
        g.vs[i]['upset']=data.loc[i,'upset']


    return g 






In [33]:
def add_round1(graph,data):
    g=graph
    for i,v in enumerate(g.vs):
        if  v['round']==1:
            vert_index=v.index

            row=data.loc[vert_index]
            # print(row)
            v1=g.add_vertex()
            v1['index']=v1.index
            v1['name']=row.top_team
            v1['seed']=row.top_seed
            v2=g.add_vertex()
            v2['index']=v2.index
            v2['name']=row.bottom_team
            v2['seed']=row.bottom_seed
            edges=[(v1.index,vert_index),(v2.index,vert_index)]
            g.add_edges(edges)
            
    return g



        
        

In [34]:
tournament.region.unique()

array(['round6', 'round5', 'West', 'South', 'Midwest', 'East'],
      dtype=object)

In [35]:
color_dict={
    'round6':'green', 
    'round5':'blue',
    'West':'yellow', 
    'South':'purple', 
    'Midwest':'orange', 
    'East':'red'
}
location_dict={
    'round6':'2,6!', 
    'round5':'2,5!',
    'West':'2,1!', 
    'South':'2,2!', 
    'Midwest':'2,3!', 
    'East':'2,4!'
}

rotate_dict={
    'round6':'TB', 
    'round5':'TB',
    'West':'RL', 
    'South':'RL', 
    'Midwest':'LR', 
    'East':'LR'
}

In [36]:
# Create a Graphviz graph
dot = graphviz.Digraph(f'March Madness Bracket {year}',comment=f'March Madness Bracket {year}', format='png' ,engine="dot")
# Set Graphviz attributes
dot.attr(label=f'March Madness Bracket {year}')
dot.attr(rankdir='RL', nodesep='.2', ranksep='.3', splines='line',size='40,40')

dot.attr('node', shape='plaintext', fixedsize='false', width='1',style='filled')
dot.graph_attr.update(landscape="false")

dot.edge_attr.update(arrowhead='none', penwidth='1')
for i,row in tournament.iterrows():

    win_team=row['winning_team']
    win_seed=row['winning_seed']
    
    team_a=row['top_team']
    team_b=row['bottom_team']

    seed_a=row['top_seed']
    seed_b=row['bottom_seed']

    rank=str(row['round'])

    next_rank=str((row['round']+1))
    
    region=row['region']

    
    dot.node(f'{team_a}{rank}',label=f'{rank}: {team_a} {int(seed_a)}',rank=rank,color=color_dict[region],group=region)
    dot.node(f'{team_b}{rank}',label=f'{rank}: {team_b} {int(seed_b)}',rank=rank,color=color_dict[region],group=region)

    dot.node(f'{win_team}{next_rank}',label=f'{next_rank}: {win_team} {int(win_seed)}',rank=next_rank,color=color_dict[region])
    cust_name=f'cluster_{region}'if region not in ['round5','round6'] else region
    with dot.subgraph(name=cust_name) as c:
        c.attr(label=f' {region}')
        
        
        if region in ['East','Midwest']:
            c.edge(f'{team_a}{rank}',f'{win_team}{next_rank}',constraint='true',samehead='true')
            c.edge(f'{team_b}{rank}',f'{win_team}{next_rank}',constraint='true',samehead='true')

        elif region in ['round5','round6']:
            c.attr(rankdir='LR' ,nodesep='.5', ranksep='.5')

            c.edge(f'{team_b}{rank}',f'{win_team}{next_rank}',constraint='true',)
            c.edge(f'{win_team}{next_rank}',f'{team_a}{rank}',constraint='true',)

        else:
            c.edge(f'{win_team}{next_rank}',f'{team_a}{rank}',constraint='true',samehead='true')
            c.edge(f'{win_team}{next_rank}',f'{team_b}{rank}',constraint='true',samehead='true')


dot.render(f'march_madness_bracket {year}', view=True)

'march_madness_bracket 2018.png'